### Tensorflow 2.0

In [1]:
import os
from glob import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline

### Hyperparameter Tuning

In [2]:
num_epochs = 5
batch_size = 32
learning_rate = 0.001
dropout_rate = 0.5
input_shape = (32,32,3)
num_classes = 10

### Build Model

In [3]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

### Data Preprocess

In [32]:
train_paths = glob('../dataset/cifar/train/*.png')
test_paths = glob('../dataset/cifar/test/*.png')

In [6]:
path = train_paths[0]
path

'../dataset/cifar/train\\0_frog.png'

In [7]:
def get_class_name(path):
    return tf.strings.regex_replace(tf.strings.split(path, '_')[-1],'.png', '')

In [8]:
class_names = [get_class_name(path) for path in train_paths]
classes = tf.unique(class_names).y.numpy()

In [9]:
def onehot_encoding(path):
    return tf.cast(classes == get_class_name(path), tf.uint8)

In [24]:
def read_dataset(path):
    # read image
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.
    
    # read label
    label = onehot_encoding(path)
    
    return image, label

In [33]:
def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [34]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(read_dataset)
train_dataset = train_dataset.map(image_preprocess) # transform images
train_dataset = train_dataset.batch(batch_size)
# train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [35]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(read_dataset)
test_dataset = test_dataset.batch(batch_size)
# train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
test_dataset = test_dataset.repeat()

### Training

In [39]:
model.fit_generator(
    train_dataset,
    steps_per_epoch=len(train_paths) // batch_size,
    validation_data=test_dataset,
    validation_steps=len(test_paths) // batch_size,
    epochs=num_epochs
)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1562/1562 [==============================] - 264s 167ms/step - loss: 1.9524 - accuracy: 0.2645 - val_loss: 1.4078 - val_accuracy: 0.4777
Epoch 2/5
1562/1562 [==============================] - 268s 171ms/step - loss: 1.4957 - accuracy: 0.4479 - val_loss: 1.3847 - val_accuracy: 0.4992
Epoch 3/5
1562/1562 [==============================] - 265s 170ms/step - loss: 1.3850 - accuracy: 0.4986 - val_loss: 1.2928 - val_accuracy: 0.5287
Epoch 4/5
1562/1562 [==============================] - 269s 172ms/step - loss: 1.3024 - accuracy: 0.5265 - val_loss: 1.2350 - val_accuracy: 0.5529
Epoch 5/5
1562/1562 [==============================] - 324s 208ms/step - loss: 1.2445 - accuracy: 0.5547 - val_loss: 1.3175 - val_accuracy: 0.5401
